In [5]:
import sys
sys.path.append("..") #Parent 
sys.path.append("../..") #grandparent
import torch
from models.components.ldm.denoiser import UNetModel
from Training_LDM.DownscalingDataModule import DownscalingDataModule
import random
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import json


In [6]:
from models.components.unet import DownscalingUnetLightning 
from models.ae_module import AutoencoderKL
from models.components.ae import SimpleConvEncoder, SimpleConvDecoder
from models.ldm_module import LatentDiffusion

In [7]:
#UNet ckpt path 
#ckpt_unet= "trained_ckpts/Training_LDM.models.components.unet.DownscalingUnetLightning_checkpoint.ckpt"
ckpt_vae= "trained_ckpts/Training_LDM.models.ae_module.AutoencoderKL_checkpoint.ckpt"
#ckpt_ldm= "trained_ckpts/LDM_checkpoint.ckpt"

In [8]:
vae_ckpt = torch.load(ckpt_vae, map_location="cpu")

print("VAE checkpoint layer shapes:")
for k, v in vae_ckpt['state_dict'].items():
    print(f"{k}: {v.shape}")

VAE checkpoint layer shapes:
encoder.net.0.proj.weight: torch.Size([64, 5, 1, 1])
encoder.net.0.proj.bias: torch.Size([64])
encoder.net.0.conv1.weight: torch.Size([64, 5, 3, 3])
encoder.net.0.conv1.bias: torch.Size([64])
encoder.net.0.conv2.weight: torch.Size([64, 64, 3, 3])
encoder.net.0.conv2.bias: torch.Size([64])
encoder.net.0.norm1.weight: torch.Size([5])
encoder.net.0.norm1.bias: torch.Size([5])
encoder.net.0.norm2.weight: torch.Size([64])
encoder.net.0.norm2.bias: torch.Size([64])
encoder.net.0.sequence.0.weight: torch.Size([5])
encoder.net.0.sequence.0.bias: torch.Size([5])
encoder.net.0.sequence.2.weight: torch.Size([64, 5, 3, 3])
encoder.net.0.sequence.2.bias: torch.Size([64])
encoder.net.0.sequence.3.weight: torch.Size([64])
encoder.net.0.sequence.3.bias: torch.Size([64])
encoder.net.0.sequence.5.weight: torch.Size([64, 64, 3, 3])
encoder.net.0.sequence.5.bias: torch.Size([64])
encoder.net.1.weight: torch.Size([64, 64, 2, 2])
encoder.net.1.bias: torch.Size([64])
decoder.net.

Instantiating the UNet

Instantiating VAE

In [9]:
encoder = SimpleConvEncoder(in_dim=5, levels=1, min_ch=64, ch_mult=1)
decoder = SimpleConvDecoder(in_dim=64, levels=2, min_ch=16)

VAE decoder needed to be softcoded to accept 4 output ch!!

In [10]:
import inspect
print(inspect.signature(SimpleConvDecoder.__init__))

(self, in_dim=1, levels=2, min_ch=64, ch_mult: int = 4)


In [11]:
model_VAE = AutoencoderKL.load_from_checkpoint(
    ckpt_vae,
    encoder=encoder,
    decoder=decoder,
    kl_weight=0.01,
    strict=False 
)

In [12]:
print(model_VAE.state_dict().keys())

odict_keys(['encoder.net.0.proj.weight', 'encoder.net.0.proj.bias', 'encoder.net.0.conv1.weight', 'encoder.net.0.conv1.bias', 'encoder.net.0.conv2.weight', 'encoder.net.0.conv2.bias', 'encoder.net.0.norm1.weight', 'encoder.net.0.norm1.bias', 'encoder.net.0.norm2.weight', 'encoder.net.0.norm2.bias', 'encoder.net.0.sequence.0.weight', 'encoder.net.0.sequence.0.bias', 'encoder.net.0.sequence.2.weight', 'encoder.net.0.sequence.2.bias', 'encoder.net.0.sequence.3.weight', 'encoder.net.0.sequence.3.bias', 'encoder.net.0.sequence.5.weight', 'encoder.net.0.sequence.5.bias', 'encoder.net.1.weight', 'encoder.net.1.bias', 'decoder.net.0.weight', 'decoder.net.0.bias', 'decoder.net.1.conv1.weight', 'decoder.net.1.conv1.bias', 'decoder.net.1.conv2.weight', 'decoder.net.1.conv2.bias', 'decoder.net.1.norm1.weight', 'decoder.net.1.norm1.bias', 'decoder.net.1.norm2.weight', 'decoder.net.1.norm2.bias', 'decoder.net.1.sequence.0.weight', 'decoder.net.1.sequence.0.bias', 'decoder.net.1.sequence.2.weight', '

In [13]:
#Key mismatch, striiping the prefix unet_regr from the keys of the Unet ckpt for compatibility

Crucial else the following ckpts wont work!!!!!!

Following code will not give a key mismatch error 

LDM ckpt loading 

In [22]:

#instantiating denoiser within the LDM
denoiser = UNetModel(
    in_channels=32,  # adjust to your latent dim
    out_channels=32,
    model_channels=64,
    num_res_blocks=2,
    attention_resolutions=[1, 2, 4],
    context_ch=None,
    channel_mult=[1, 2, 4, 4],
    conv_resample=True,
    dims=2,
    use_fp16=False,
    num_heads=4
)



In [23]:
ckpt_ldm= "trained_ckpts/LDM_checkpoint.ckpt"
ldm_ckpt = torch.load(ckpt_ldm, map_location="cpu")

In [24]:
#instantiating LDM
model_LDM = LatentDiffusion(
    denoiser=denoiser,
    autoencoder=model_VAE,
    timesteps=1000,#torch.tensor for reference later
    beta_schedule="linear",
    loss_type="l2",
    use_ema=True,
    lr=1e-4,
    lr_warmup=0,
    linear_start=1e-4,
    linear_end=2e-2,
    cosine_s=8e-3,
    parameterization="eps"
)


In [19]:
#The keys within LDM ckpt also have to be renamed for compatibiltiy 

In [25]:
remapped_ldm_state_dict = {}
for k, v in ldm_ckpt["state_dict"].items():
    # Remove both 'autoencoder.unet_regr.unet.' and 'autoencoder.unet_regr.' prefixes
    if k.startswith("autoencoder.unet_regr.unet."):
        new_key = "autoencoder.unet." + k[len("autoencoder.unet_regr.unet."):]
    elif k.startswith("autoencoder.unet_regr."):
        new_key = "autoencoder.unet." + k[len("autoencoder.unet_regr."):]
    else:
        new_key = k
    remapped_ldm_state_dict[new_key] = v

In [26]:
model_LDM.load_state_dict(remapped_ldm_state_dict, strict=False)

<All keys matched successfully>

In [27]:
#Expected vs real key 
print(model_LDM.state_dict().keys())
print(remapped_ldm_state_dict.keys())

odict_keys(['betas', 'alphas_cumprod', 'alphas_cumprod_prev', 'sqrt_alphas_cumprod', 'sqrt_one_minus_alphas_cumprod', 'denoiser.time_embed.0.weight', 'denoiser.time_embed.0.bias', 'denoiser.time_embed.2.weight', 'denoiser.time_embed.2.bias', 'denoiser.input_blocks.0.0.weight', 'denoiser.input_blocks.0.0.bias', 'denoiser.input_blocks.1.0.in_layers.2.weight', 'denoiser.input_blocks.1.0.in_layers.2.bias', 'denoiser.input_blocks.1.0.emb_layers.1.weight', 'denoiser.input_blocks.1.0.emb_layers.1.bias', 'denoiser.input_blocks.1.0.out_layers.3.weight', 'denoiser.input_blocks.1.0.out_layers.3.bias', 'denoiser.input_blocks.2.0.in_layers.2.weight', 'denoiser.input_blocks.2.0.in_layers.2.bias', 'denoiser.input_blocks.2.0.emb_layers.1.weight', 'denoiser.input_blocks.2.0.emb_layers.1.bias', 'denoiser.input_blocks.2.0.out_layers.3.weight', 'denoiser.input_blocks.2.0.out_layers.3.bias', 'denoiser.input_blocks.3.0.op.weight', 'denoiser.input_blocks.3.0.op.bias', 'denoiser.input_blocks.4.0.in_layers.2.w

Extra key above, rest matching, not giving an issue if strict=False

In [28]:
dm = DownscalingDataModule(
    train_input={
        "precip": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/RhiresD_input_test_chronological_scaled.nc",
        "temp": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TabsD_input_test_chronological_scaled.nc",
        "temp_min": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TminD_input_test_chronological_scaled.nc",
        "temp_max": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TmaxD_input_test_chronological_scaled.nc"
    },
    train_target={
        "precip": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/RhiresD_target_test_chronological_scaled.nc",
        "temp": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TabsD_target_test_chronological_scaled.nc",
        "temp_min": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TminD_target_test_chronological_scaled.nc",
        "temp_max": "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TmaxD_target_test_chronological_scaled.nc"
    },
    elevation="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/elevation.tif",
    batch_size=32,
    num_workers=4,
    preprocessing={
        "variables": {
            "input": {
                "precip": "RhiresD",
                "temp": "TabsD",
                "temp_min": "TminD",
                "temp_max": "TmaxD"
            },
            "target": {
                "precip": "RhiresD",
                "temp": "TabsD",
                "temp_min": "TminD",
                "temp_max": "TmaxD"
            }
        },
        "preprocessing": {
            "nan_to_num": True,
            "nan_value": 0.0
        }
    }
)

dm.setup()


denorming functions along with parameter files 

In [29]:
with open("../Training_Chronological_Dataset/RhiresD_scaling_params_chronological.json") as f:
    pr_params = json.load(f)
with open("../Training_Chronological_Dataset/TabsD_scaling_params_chronological.json") as f:
    temp_params = json.load(f)
with open("../Training_Chronological_Dataset/TminD_scaling_params_chronological.json") as f:
    temp_min_params = json.load(f)
with open("../Training_Chronological_Dataset/TmaxD_scaling_params_chronological.json") as f:
    temp_max_params = json.load(f)

def denorm_pr(x):
    # Min-max
    return x * (pr_params["max"] - pr_params["min"]) + pr_params["min"]

def denorm_temp(x, params):
    # z
    return x * params["std"] + params["mean"]



In [36]:
# Get a batch from the dataloader
train_loader = dm.train_dataloader()
train_batch = next(iter(train_loader))
train_inputs, train_targets = train_batch

# Select a random sample
idx = random.randint(0, train_inputs.shape[0] - 1)
input_sample = train_inputs[idx].unsqueeze(0)  # (1, channels, H, W)



In [ ]:
from models.components.ldm.denoiser.ddim import DDIMSampler

# Rand frame
train_loader = dm.train_dataloader()
train_batch = next(iter(train_loader))
train_inputs, train_targets = train_batch
idx = random.randint(0, train_inputs.shape[0] - 1)
input_sample = train_inputs[idx].unsqueeze(0)  # (1, ch, H, W)

with torch.no_grad():
    # LAtent
    mean, log_var = model_VAE.encode(input_sample)
    std = torch.exp(0.5 * log_var)
    eps = torch.randn_like(std)
    latent = mean + eps * std  # stochastic sampling from a Gaussian

    sampler = DDIMSampler(model_LDM)
    # shape for single sample
    shape = latent.shape[1:]
    samples, _ = sampler.sample(
        S=50,  # denoising steps
        batch_size=1,
        shape=shape,
        x_T=latent,  # start from encoded latent
        eta=0.2,     # 0.0 for deterministic, >0 for stochastic
        verbose=False,
        progbar=True
    )

    # Decode
    recon = model_VAE.decode(samples)

images_ldm = [
    denorm_pr(input_sample[0, 0].cpu().numpy()),
    denorm_pr(recon[0, 0].cpu().numpy()),       
    denorm_temp(input_sample[0, 1].cpu().numpy(), temp_params),   
    denorm_temp(recon[0, 1].cpu().numpy(), temp_params),           
    denorm_temp(input_sample[0, 2].cpu().numpy(), temp_min_params),
    denorm_temp(recon[0, 2].cpu().numpy(), temp_min_params),      
    denorm_temp(input_sample[0, 3].cpu().numpy(), temp_max_params),
    denorm_temp(recon[0, 3].cpu().numpy(), temp_max_params),       
]
titles_ldm = [
    "Input precip", "LDM Output precip",
    "Input temp", "LDM Output temp",
    "Input temp_min", "LDM Output temp_min",
    "Input temp_max", "LDM Output temp_max"
]
 
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
for ax, img, title in zip(axes.flatten(), images_ldm, titles_ldm):
    ax.imshow(img, cmap='viridis')
    ax.set_title(title)
    ax.axis('off')
plt.tight_layout()
plt.show()

Data shape for DDIM sampling is (1, 32, 120, 185), eta 0.2
Running DDIM Sampling with 50 timesteps


DDIM Sampler:  66%|██████▌   | 33/50 [31:35<16:07, 56.88s/it]